In [51]:
import json

In [65]:
import osmnx as ox
import numpy as np
import networkx as nx
import re

with open("data/Gemeinde Höchst/guideposts_coordinates.txt") as f:
    lines = f.readlines()

    gps_coordinates = []

    G = ox.graph_from_place("Gemeinde Höchst", network_type='all_private', simplify=False)

    pattern = r'Latitude: (\d+\.\d+), Longitude: (\d+\.\d+),.*'

    for line in lines:
        lat = float(re.sub(pattern, r'\1', line))
        lon = float(re.sub(pattern, r'\2', line))
        gps_coordinates.append((lat, lon))

    # Find the nearest nodes to the GPS coordinates
    nodes = ox.distance.nearest_nodes(G, X=[x[1] for x in gps_coordinates], Y=[x[0] for x in gps_coordinates])

    # Calculate the shortest path length between each pair of nodes
    n = len(nodes)
    distance_matrix = np.zeros((n, n), dtype=int)

    for i in range(n):
        for j in range(n):
            if i != j:
                try:
                    distance_matrix[i][j] = int(nx.shortest_path_length(G, nodes[i], nodes[j], weight='length'))
                except nx.NetworkXNoPath:
                    distance_matrix[i][j] = 999999999

distance_matrix

array([[   0,  114, 1314, ...,   31, 1424,  212],
       [ 114,    0, 1429, ...,  146, 1534,  321],
       [1314, 1429,    0, ..., 1282, 2426, 1473],
       ...,
       [  31,  146, 1282, ...,    0, 1416,  204],
       [1424, 1534, 2396, ..., 1416,    0, 1212],
       [ 212,  321, 1459, ...,  204, 1212,    0]])

In [66]:
data = {"distance_matrix": distance_matrix, "num_vehicles": 1, "depot": 4}

def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()} miles")
    index = routing.Start(0)
    plan_output = "Route for vehicle 0:\n"
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += f" {manager.IndexToNode(index)} ->"
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += f" {manager.IndexToNode(index)}\n"
    print(plan_output)
    plan_output += f"Route distance: {route_distance}miles\n"

In [67]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(
    len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
)

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)


def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data["distance_matrix"][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    print_solution(manager, routing, solution)

Objective: 23224 miles
Route for vehicle 0:
 4 -> 3 -> 2 -> 31 -> 0 -> 29 -> 1 -> 5 -> 23 -> 30 -> 6 -> 7 -> 28 -> 24 -> 25 -> 26 -> 8 -> 27 -> 13 -> 9 -> 10 -> 11 -> 12 -> 14 -> 15 -> 16 -> 19 -> 18 -> 17 -> 32 -> 20 -> 22 -> 21 -> 33 -> 4
